
# DeepAgents Validation 
A minimal, **runnable** mock of a LangChain DeepAgents-style workflow to validate data output and step ordering.

- Step **7 = Diagnostics** (runs first)
- Step **8 = Run-All Validation** (runs after diagnostics)
- No duplicate `__main__` blocks
- All functions defined cohesively within single cells (no indentation breakage)


In [1]:

from __future__ import annotations
from dataclasses import dataclass, field
from typing import List, Dict, Any, Optional


In [2]:

@dataclass
class Message:
    role: str
    content: str


In [3]:

@dataclass
class MockAgent:
    name: str
    memory: Dict[str, Any] = field(default_factory=dict)

    def _get_thread(self, config: Optional[Dict[str, Any]] = None) -> str:
        cfg = (config or {}).get("configurable", {})
        return cfg.get("thread_id", "default-thread")

    def invoke(self, payload: Dict[str, Any], config: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        thread = self._get_thread(config)
        msgs: List[Message] = [
            Message(**m) if not isinstance(m, Message) else m
            for m in payload.get("messages", [])
        ]

        # Super simple behavior & memory
        last_user = next((m for m in reversed(msgs) if m.role == "user"), None)

        # Memory store/recall
        if last_user and "favorite topic is" in last_user.content.lower():
            topic = last_user.content.split("is", 1)[-1].strip().rstrip(".")
            self.memory.setdefault(thread, {})["favorite_topic"] = topic
            reply = f"Noted. I'll remember your favorite topic is {topic}."
        elif last_user and "what's my favorite topic" in last_user.content.lower():
            topic = self.memory.get(thread, {}).get("favorite_topic", "unknown")
            reply = f"You told me your favorite topic is {topic}."
        elif last_user and "plan:" in last_user.content.lower():
            # Make a tiny plan
            steps = ["Research requirement", "Draft outline", "Implement", "Test", "Deliver"]
            reply = "Plan generated:\n" + "\n".join(f"{i+1}. {s}" for i, s in enumerate(steps))
        elif last_user and "search(" in last_user.content.lower():
            # mock search tool usage
            reply = "Search results: ['doc_a', 'doc_b', 'doc_c']"
        else:
            reply = f"Hello from {self.name}. You said: {last_user.content if last_user else '(no input)'}"

        # Correctly indented return block
        result_msgs = msgs + [Message(role="assistant", content=reply)]
        return {
            "messages": [m.__dict__ for m in result_msgs],
            "state": {"thread_id": thread, "memory": self.memory.get(thread, {})},
        }


In [4]:

@dataclass
class MockCoordinator:
    planner: MockAgent
    researcher: MockAgent
    implementer: MockAgent

    def invoke(self, payload: Dict[str, Any], config: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        # very naive: planner -> researcher -> implementer
        thread = (config or {}).get("configurable", {}).get("thread_id", "default-thread")
        messages = payload.get("messages", [])

        p = self.planner.invoke({"messages": messages + [Message(role="user", content="Plan: build a demo").__dict__]}, config)
        r = self.researcher.invoke({"messages": p["messages"] + [Message(role="user", content="search(query='demo')").__dict__]}, config)
        i = self.implementer.invoke({"messages": r["messages"] + [Message(role="user", content="Implement based on findings.").__dict__]}, config)

        return i


In [5]:

def make_basic_agent(name: str = "basic-agent") -> MockAgent:
    return MockAgent(name=name)

def _pretty_last(messages: List[Dict[str, Any]]) -> str:
    if not messages:
        return "(no messages)"
    return messages[-1].get("content", "(no content)")


## Tests 2–6

In [6]:

# 2. CREATE A BASIC AGENT
def test_basic_agent() -> MockAgent:
    agent = make_basic_agent()
    result = agent.invoke({"messages": [Message(role="user", content="Hello").__dict__]})
    assert "messages" in result and isinstance(result["messages"], list)
    assert result["messages"][-1]["role"] == "assistant"
    print("✓ Basic agent returns assistant message")
    return agent

# 3. TEST SUBAGENTS
def test_subagents() -> MockCoordinator:
    planner = make_basic_agent("planner")
    researcher = make_basic_agent("researcher")
    implementer = make_basic_agent("implementer")
    coord = MockCoordinator(planner=planner, researcher=researcher, implementer=implementer)

    out = coord.invoke({"messages": [Message(role="user", content="Build something cool.").__dict__]})
    assert isinstance(out, dict) and "messages" in out
    print("✓ Subagents coordinator returns final messages")
    return coord

# 4. TEST BACKEND CUSTOMIZATION
def test_custom_backend() -> MockAgent:
    # In real LC DeepAgents you'd switch LLM/tool backends; we just rename for visibility
    agent = make_basic_agent(name="custom-backend-agent")
    res = agent.invoke({"messages": [Message(role="user", content="Hello custom").__dict__]})
    assert res["messages"][-1]["role"] == "assistant"
    print("✓ Custom backend stub works")
    return agent

# 5. TEST PLANNING TOOL
def test_planning_tool() -> MockAgent:
    agent = make_basic_agent("planner")
    res = agent.invoke({"messages": [Message(role="user", content="Plan: Ship a feature").__dict__]})
    assistant_text = res["messages"][-1]["content"]
    assert "Plan generated:" in assistant_text
    print("✓ Planning generated a sequence of steps")
    return agent

# 6. TEST MEMORY PERSISTENCE
def test_memory_persistence() -> MockAgent:
    agent = make_basic_agent("memory-agent")
    config = {"configurable": {"thread_id": "test-thread-1"}}

    # First: store preference
    agent.invoke({"messages": [Message(role="user", content="My favorite topic is AI agents.").__dict__]}, config)
    # Second: recall
    res = agent.invoke({"messages": [Message(role="user", content="What's my favorite topic?").__dict__]}, config)

    assert "AI agents" in res["messages"][-1]["content"]
    assert res["state"]["memory"].get("favorite_topic") == "AI agents"
    print("✓ Memory persists across thread using config.thread_id")
    return agent


## 7. Diagnostics (runs first)

In [7]:

def run_diagnostics() -> None:
    print("\n=== Diagnostic: Basic Agent ===")
    a = test_basic_agent()
    r = a.invoke({"messages": [Message(role="user", content="Echo me please").__dict__]})
    print("Last:", _pretty_last(r["messages"]))

    print("\n=== Diagnostic: Subagents ===")
    c = test_subagents()
    sub = c.invoke({"messages": [Message(role="user", content="Do multi-step").__dict__]})
    print("Last:", _pretty_last(sub["messages"]))

    print("\n=== Diagnostic: Planning ===")
    p = test_planning_tool()
    plan = p.invoke({"messages": [Message(role="user", content="Show the plan again.").__dict__]})
    print("Last:", _pretty_last(plan["messages"]))

    print("\n=== Diagnostic: Memory ===")
    m = test_memory_persistence()
    recall = m.invoke({"messages": [Message(role="user", content="What's my favorite topic?").__dict__]}, {"configurable": {"thread_id": "test-thread-1"}})
    print("Last:", _pretty_last(recall["messages"]))

    print("\nDiagnostics complete.")


## 8. Run All Validation (runs after diagnostics)

In [8]:

def run_validation() -> None:
    print("\n" + "="*60)
    print("🧠 Deep Agents Validation Suite")
    print("="*60)
    try:
        test_basic_agent()
        test_subagents()
        test_custom_backend()
        test_planning_tool()
        test_memory_persistence()
        print("\n" + "="*60)
        print("✅ All validation tests completed successfully!")
        print("="*60)
    except Exception as e:
        print(f"\n❌ Validation failed: {e}")
        raise


### Execute

In [9]:
if __name__ == "__main__":
    run_diagnostics()
    run_validation()


=== Diagnostic: Basic Agent ===
✓ Basic agent returns assistant message
Last: Hello from basic-agent. You said: Echo me please

=== Diagnostic: Subagents ===
✓ Subagents coordinator returns final messages
Last: Hello from implementer. You said: Implement based on findings.

=== Diagnostic: Planning ===
✓ Planning generated a sequence of steps
Last: Hello from planner. You said: Show the plan again.

=== Diagnostic: Memory ===
✓ Memory persists across thread using config.thread_id
Last: You told me your favorite topic is AI agents.

Diagnostics complete.

🧠 Deep Agents Validation Suite
✓ Basic agent returns assistant message
✓ Subagents coordinator returns final messages
✓ Custom backend stub works
✓ Planning generated a sequence of steps
✓ Memory persists across thread using config.thread_id

✅ All validation tests completed successfully!
